In [2]:
!pip install crewai==0.95.0
!pip install crewai-tools==0.25.8
!pip install duckduckgo-search
!pip install langchain-community
!pip install litellm

In [3]:
from crewai import Agent, Task
from crewai.tools import tool
from crewai.llm import LLM
from langchain_community.tools import DuckDuckGoSearchRun
from datetime import datetime, timezone
import os

# Current date for context
Now = datetime.now()
Today = Now.strftime("%d-%b-%Y")
print(Today)

# Define a web search tool
@tool("DuckDuckGo Search")
def search_tool(search_query: str):
    """Search the internet for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

11-Jul-2025


In [4]:
with open(".env", "w") as f:
    f.write("EXA_API_KEY=*YOUR_K*\n")
    f.write("OPENROUTER_API_KEY=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")

In [5]:
from crewai_tools import EXASearchTool
import os

from dotenv import load_dotenv
load_dotenv()

EXA_API_KEY = os.getenv("EXA_API_KEY")
OPENROUTER_KEY = os.getenv("OPENROUTER_API_KEY")

Exa_Search_Tool = EXASearchTool ()

C:\Users\saket.khopkar\anaconda3\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
C:\Users\saket.khopkar\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\saket.khopkar\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 45.0.0.
  "class": algorithms.Blowfish,
C:\Use

In [6]:
from crewai import Agent

content_explorer = Agent (
    role = 'content explorer',
    goal = 'Gather and provide latest information about the topic from internet',
    # llm = 'gemini/gemini-2.0-flash-lite',
    # llm = 'groq/gemma2-9b-it',
    llm = 'openrouter/deepseek/deepseek-chat-v3-0324:free',
    # llm = 'gpt-4o',
    verbose = True,
    backstory = ('You are an expert researcher, who can gather detailed information about a topic.\
                  Gather at least 10 information. Consider you are on : '+Today),
    tools = [Exa_Search_Tool],
    cache = True,
    max_iter = 5
)

Script_Writer = Agent (
    role = 'Script Writer',
    goal = 'With the details given to you create an interesting conversational script out of it',
    llm = 'openrouter/deepseek/deepseek-chat-v3-0324:free',
    # llm=script_llm,
    verbose = True,
    backstory = ('You are an expert in Shakesperean literature. You are very good in creating conversations with the given chain of information.\
                 Tell as a script in 200 words.\
                 Consider you are on '+Today),
)

In [7]:
from crewai import Task

# Task to gather Latest information
get_details = Task (
                      description = "Get latest, trending, interesting information and news about {topic}",
                      expected_output = "Latest news, interesting information and trivia about {topic}",
                      agent = content_explorer
                    )

## Task to create script.
create_a_script = Task (
                      description = "Considering the given details in time order make an interesting conversation",
                      expected_output = "A humorous conversation connecting key details",
                      agent = Script_Writer,
                      context = [get_details],
                      output_file = 'Script.txt'
                    )

In [8]:
from crewai import Crew, Process
from datetime import datetime

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew (
    agents = [content_explorer, Script_Writer],
    tasks = [get_details, create_a_script],
    verbose = True,
    Process = Process.sequential,
    step_callback = timestamp
)

In [9]:
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
result = crew.kickoff (inputs={'topic' : "GTA 6"})

# Agent: content explorer
## Task: Get latest, trending, interesting information and news about GTA 6
2025-07-11 11:49:20.877587


# Agent: content explorer
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"latest news GTA 6 July 2025\"}"
## Tool Output: 

Search results: 

2025-07-11 11:49:20.877587


2025-07-11 11:50:15,885 - 18776 - llm.py-llm:187 - ERROR: LiteLLM call failed: litellm.APIError: APIError: OpenrouterException - {"error":{"message":"Internal Server Error","code":500}}




LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.

# Agent: content explorer
## Task: Get latest, trending, interesting information and news about GTA 6
2025-07-11 11:50:22.048561


# Agent: content explorer
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"latest news about GTA 6 July 2025\"}"
## Tool Output: 

Search results: 

2025-07-11 11:50:22.048561
2025-07-11 11:50:41.240157


# Agent: content explorer
## Final Answer: 
Here are the latest updates and interesting details about **GTA 6** as of July 2025:  

1. **Release Date**: Officially set for **Fall 2025**, with pre-orders opening in September 2025.  

2. **Setting**: The game returns to **Vice City**, expanded with new regions like swamps, islands, and rural areas.  

3. **Dual Protagonists**: Features **Jason and Lucia**—the first female lead in a mainline GTA game.  

4. **Massive Map**: Largest GTA map ever, including underwater exploration.  

5. **Live Service Plans